# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd

## 0. Lectura del fichero

In [2]:
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

In [3]:
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


## 1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [10]:
df_1 = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == "Yes")]

In [12]:
df_1.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


In [15]:
# Comprobacion
df_1['Total Claim Amount'].mean()
df_1['Response'].unique()

array(['Yes'], dtype=object)

Hay 26 personas que hayan respondido si a la ultima campaña con un total claim amount mayor que 1000

## 2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [16]:
df_2 = df[df['Response'] == 'Yes']

In [18]:
df_2.groupby(['Policy Type', 'Gender']).agg({'Total Claim Amount' : 'mean'})

Total Claim Amount
Policy Type    Gender                    
Corporate Auto F               433.738499
               M               408.582459
Personal Auto  F               452.965929
               M               457.010178
Special Auto   F               453.280164
               M               429.527942

Podemos observar como la mayor diferencia entre géneros es en el tipo de póliza `Corporate Auto`. Mientras que en el resto están bastante igulados entre géneros, siendo en el tipo de póliza `Persona Auto` prácticamente la misma. También, podemos destacar que a diferencia de lo que se podría pensar previamente en los clientes con un tipo de póliza `Personal Auto` superan en media a aquellos que tienen un tipo de póliza `Special Auto`. 

## 3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [21]:
df_3 = df.groupby('State').agg({'Customer': 'count'})

In [22]:
df_3

,Customer
State,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


In [24]:
df_3[df_3['Customer'] > 500]

,Customer
State,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


## 4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [27]:
df_4 = df.groupby(['Education', 'Gender']).agg(
    {'Customer Lifetime Value': ['max', 'min', 'median' ]}
)

In [28]:
df_4

Customer Lifetime Value                          
                                                max          min       median
Education            Gender                                                  
Bachelor             F                  73225.95652  1904.000852  5640.505303
                     M                  67907.27050  1898.007675  5548.031892
College              F                  61850.18803  1898.683686  5623.611187
                     M                  61134.68307  1918.119700  6005.847375
Doctor               F                  44856.11397  2395.570000  5332.462694
                     M                  32677.34284  2267.604038  5577.669457
High School or Below F                  55277.44589  2144.921535  6039.553187
                     M                  83325.38119  1940.981221  6286.731006
Master               F                  51016.06704  2417.777032  5729.855012
                     M                  50568.25912  2272.307310  5579.099207

Podemos observar como la mayor diferencia entre el nivel de eduación se da en `Bachelor`. Mientras que la mayor diferencia enyre hombre y mujer la vemos en un nivel de  `High School or Below` siendo mayor la del género masculino. También, cabe destacar que uno de los valores más bajos en el máximo es para aquellas personas que posean un nivel de eduación de `Doctor`.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [30]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [36]:
df_mod = df

In [37]:
df_mod['Effective To Date'] = pd.to_datetime(df_mod['Effective To Date'])

C:\Users\Carlota\AppData\Local\Temp\ipykernel_11684\1021659570.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_mod['Effective To Date'] = pd.to_datetime(df_mod['Effective To Date'])


In [38]:
df_mod['Effective To Date']

0       2011-02-18
1       2011-01-18
2       2011-02-10
3       2011-01-11
4       2011-01-17
           ...    
10905   2011-01-19
10906   2011-01-06
10907   2011-02-06
10908   2011-02-13
10909   2011-01-08
Name: Effective To Date, Length: 10910, dtype: datetime64[ns]

In [39]:
df_mod['Month'] = df_mod['Effective To Date'].dt.strftime('%B')  # Nombre del mes
df_mod['Year'] = df_mod['Effective To Date'].dt.year  # Año


In [40]:
df_mod['Month']

0        February
1         January
2        February
3         January
4         January
           ...   
10905     January
10906     January
10907    February
10908    February
10909     January
Name: Month, Length: 10910, dtype: object

In [42]:
df_mod.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type', 'Month', 'Year'],
      dtype='object')

In [43]:
df_5 = pd.pivot_table(df_mod, values = 'Customer', 
                      index='State', columns= 'Month', aggfunc='count')

In [44]:
df_5

Month,February,January
State,,
Arizona,929,1008
California,1634,1918
Nevada,442,551
Oregon,1344,1565
Washington,425,463


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [48]:
df_6 = df_mod.groupby(['State', 'Month']).agg({'Customer': 'count'}).reset_index()

In [49]:
df_6

,State,Month,Customer
0,Arizona,February,929
1,Arizona,January,1008
2,California,February,1634
3,California,January,1918
4,Nevada,February,442
5,Nevada,January,551
6,Oregon,February,1344
7,Oregon,January,1565
8,Washington,February,425
9,Washington,January,463


In [50]:
policy_by_state = df_6.groupby('State')['Customer'].sum().reset_index()

In [54]:
total_states = policy_by_state.sort_values(by="Customer", ascending= False).head(3)

In [55]:
total_states

,State,Customer
1,California,3552
3,Oregon,2909
0,Arizona,1937


In [57]:
list_states = total_states['State'].tolist()
list_states

['California', 'Oregon', 'Arizona']

In [61]:
df_6[df_6['State'].isin(list_states)]

,State,Month,Customer
0,Arizona,February,929
1,Arizona,January,1008
2,California,February,1634
3,California,January,1918
6,Oregon,February,1344
7,Oregon,January,1565


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [65]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type', 'Month', 'Year'],
      dtype='object')

In [76]:
melted_df = pd.melt(df, id_vars = ['Customer', 'Response'], 
                    value_vars=['Sales Channel', 'Renew Offer Type'], 
                    var_name = 'Marketing Channel', 
                    value_name= 'Channel Value')

In [77]:
respuestas_si = melted_df[melted_df['Response'] == 'Yes']
respuestas_si

,Customer,Response,Marketing Channel,Channel Value
3,XL78013,Yes,Sales Channel,Branch
8,FM55990,Yes,Sales Channel,Branch
15,CW49887,Yes,Sales Channel,Branch
19,NJ54277,Yes,Sales Channel,Call Center
27,MQ68407,Yes,Sales Channel,Agent
...,...,...,...,...
21754,FM31768,Yes,Renew Offer Type,Offer2
21762,KZ80424,Yes,Renew Offer Type,Offer2
21782,XT67997,Yes,Renew Offer Type,Offer2
21797,BY78730,Yes,Renew Offer Type,Offer1


External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here